In [49]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Para visualización de datos
# -----------------------------------------------------------------------


# Para cargar los datos
# -----------------------------------------------------------------------
import pickle

# Para sistemas de recomendacion basados en cotenido
# -----------------------------------------------------------------------
from scipy.spatial.distance import pdist , squareform   # Para calcular la distancia entre puntos 

# Importamos nuestras funciones
# -----------------------------------------------------------------------


# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
import psycopg2

In [45]:
!pip install scipy

  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)


In [59]:
conn = psycopg2.connect(
    dbname='prueba_tecnica',
    user='postgres',
    password='admin',
)

sql_query = """
SELECT 
    p.generos AS genero,
    d.cali_imdb AS calificacion,
    RANK() OVER (ORDER BY d.cali_imdb DESC) AS ranking,
    d.titulo
FROM 
    peliculas p
JOIN 
    detalles_peliculas d
ON 
    p.index = d.index;
"""

cursor = conn.cursor()
cursor.execute(sql_query)
resultados = cursor.fetchall()

In [60]:
df = pd.DataFrame(resultados)

In [61]:
df.dropna(inplace=True)

In [62]:
df

,0,1,2,3
492,Horror,9.8,493,Hurst
493,Thriller,9.6,494,Graveyard of Dreams
494,Horror,9.6,494,No Return
495,Horror,9.5,496,Koritsia tis thalassas
496,Thriller,9.4,497,Madam Trigger
...,...,...,...,...
1448,Thriller,2.1,1449,Primavera di granito
1449,Horror,2.0,1450,Texas Death Trippin'
1450,Horror,1.9,1451,12/12/2012
1451,Romance,1.7,1452,Prem Aggan


In [64]:
df.reset_index(inplace=True)

In [65]:
df.drop("index", inplace=True, axis=1)
df.drop(2, inplace=True, axis=1)

In [66]:
df.rename(columns={0:"Genero"}, inplace=True)
df.rename(columns={1:"Puntuacion", 3:"Titulo"}, inplace=True)

In [68]:
df.drop("level_0", axis=1, inplace=True)

In [70]:
df.set_index("Genero", inplace=True)

In [69]:
df

,Genero,Puntuacion,Titulo
0,Horror,9.8,Hurst
1,Thriller,9.6,Graveyard of Dreams
2,Horror,9.6,No Return
3,Horror,9.5,Koritsia tis thalassas
4,Thriller,9.4,Madam Trigger
...,...,...,...
956,Thriller,2.1,Primavera di granito
957,Horror,2.0,Texas Death Trippin'
958,Horror,1.9,12/12/2012
959,Romance,1.7,Prem Aggan


In [71]:
userRatings = df.pivot_table(index=['Genero'],columns=['Titulo'],values='Puntuacion').fillna(0)
userRatings.head()

Titulo,'Ne gÃ¼nstige Gelegenheit,12 Feet Deep,12/12/2012,13 segundos,14 Days in Paradise,18 anni tra una settimana,25Gs,3 Chinesen mit dem Kontrabass,3 Days of Darkness,3:15 AM,5,6079_Zn,7,A Beautiful Mess,A Good Dream,A Haunting at The Octagon Mansion,A Little Bit of Lipstick,A Matter of Degrees,A Rainy Day,A Shameless Revenge,A Smile Like Yours,A Taste,A Vigilante,A Waiter Tomorrow,A kis utazÃ¡s,A praga,Aa Intlo,Ab... Bas!,About A Short Film,Accused at 17,Achanak,Achoura,Adam & Eva,Afflatus,Afghan Knights,After the Third Bell,Aftermath: A Test of Love,Agatha,Agents of Change: Project Polymer,AibÃ´: GekijÃ´-ban IV,Akala mo...,Ako si Ninoy,All Flowers in Time,All My Life,All the Fists in the World Won't Save You Now,Almost Heaven,Alternate,Alyti,Am I,American Exorcist,American Short,Among Thieves,Amor que mata,An Idle Mind Is the Devil's Playground,An Unexpected Love,Anakiseuteu Anarchists,Angaaray,Annabelle: Creation,Annabellum: The Curse of Salem,Anomie,Anonymous Vol. 1 - The Dreamland Adventures the Mysteries of Atlantis,Anti Cupido,Any Way the Wind Blows,Apocalypse Road,Apostle,Apparition,Armugan,Aryan: Unbreakable,Asbury Park,Askin GÃ¶nÃ¼l Yazisi,Astro,Aswang: A Journey Into Myth,Attack of the Adult Babies,Aux,Awaken,Azaad Desh Ke Gulam,BIR,Bacao,Bakom Snart Ã¤r det lÃ¶rdag igen,Ban khang winyan,Bandwagon,Bangkok Dangerous,Bank Robber,Bannerghatta,Batman: Dead End,Bats in the Belfry,Bbuddah... Hoga Terra Baap,Be-seu-teu-sel-leo,Beat,Ben: The Movie,Bernard and Doris,Between the Trees,Bhoner: The Movie,Bigfoot,Birthday,Bit Players,Black Pond,Black Pumpkin,Blarghaaahrgarg,Blindsided,Blood Vow,Blue,Boarding School,Bob Roberts,Bobby,Body Bags,Bolse vita,Bonnie & Clyde vs. Dracula,Boogeyman Pop,"Boonie Bears, to the Rescue!",Boy Friend,Boy Meets Girl Stories #4: Geen antwoord,BoyTown,Boychick,Brightheart,Bun siu hai,Buster's Mal Heart,Buttf**ker,Caliche Run,Call Me Claus,Calling,Camp X-Ray: Ghosts of GuantÃ¡namo Bay,Captive Heart: The James Mink Story,Casablanca,Cassette,Caustic Zombies,Ce jour-lÃ,Cerro Torre: Schrei aus Stein,Cesta peklem,Chaco,Charlie St. Cloud,Cheun,Cheuuat gaawn chim,Chi bi: Jue zhan tian xia,Children of the Grave 2,Chingaari,Chong qing ai qing gan jue,"Chot [Aaj Isko, Kal Tereko]",Christian Blake,Christmas Twister,Christmas on Division Street,Chuck Steel: Night of the Trampires,Churuli,Circles,Class Savage,Client Seduction,CobraGator,Color-Blinded,Como Nascem os Anjos,Company K,Cowboy,Creature,Criss Angel Mindfreak,Criss Angel Supernatural,Crna ovca u restoranu za zombije,Crumbs,Cul de sac,Cumbia callera,Curse of Mesopotamia,Daava,Daddy Cross,Dagaalty,Dai zek lo,Dante's View,Dark Heart,Darkness Reigns,Darkwood Manor,Dawna of the Dead,Day 66,De zaak Alzheimer,Dead Again,Dead Ahead,Dead Bodies Everywhere,Dead of Winter,Deadtime,Dear Father,"Death, Deceit & Destiny Aboard the Orient Express",Demons,Den Japanska Illusionen,Den sorte celle,Der Traum vom SÃ¼den,Der Wien-Krimi: Blind ermittelt - Tod an der Donau,Der Zauber des Regenbogens,Detektor,Devil's Pond,Devot,Dibbuk,Die Mediocren,Die Waschmaschine,Die andere Frau,Die: Lianhua,Diljale,Dinner with Friends,DirektÃ¸ren for det hele,Distants,Distorted,Divar,Dok,Doli Saja Ke Rakhna,Dolphins,Don't Look Under the Bed,Don't Look in the Basement,Dontel papa,Doraemon: Nobita to fushigi kazetsukai,Doug's 1st Movie,Dr. Munshir Diary,Dream a Little Dream for Me,DristiBhram,Dubai,Duo ming jie chu,Easter Bunny Bloodbath,Eat Me,Eating Out,Eccezzziunale veramente - Capitolo secondo... me,Ecco fatto,Echoes of the Passed,Eddie,Ein Sack voll Geld,Ek: The Power of One,El ataque de los robots de nebulosa-5,El bar,El corazÃ³n de la tierra,El domini del sentits,El grito en el cielo,El que habita las alturas,El sol,El sueÃ±o de Ibiza,Elaan,Ellos Volvieron,Elsa y Fred,EmpanÃ¡ de pino,Encrucijada,End of the Line,Endoresu warutsu,Enemy Within,Enfants terribles,"English, August",Epoch: Evolution,Erakkorapu,Escape Room,Eskiya,Estrela Radiante,"Eternal Coma, 1994",Ex Abyssus,Exile,Exit 1

In [72]:
# calculamos la matriz de correlacio
matr_corre = userRatings.corr(method='pearson')
matr_corre.head()

Titulo,'Ne gÃ¼nstige Gelegenheit,12 Feet Deep,12/12/2012,13 segundos,14 Days in Paradise,18 anni tra una settimana,25Gs,3 Chinesen mit dem Kontrabass,3 Days of Darkness,3:15 AM,5,6079_Zn,7,A Beautiful Mess,A Good Dream,A Haunting at The Octagon Mansion,A Little Bit of Lipstick,A Matter of Degrees,A Rainy Day,A Shameless Revenge,A Smile Like Yours,A Taste,A Vigilante,A Waiter Tomorrow,A kis utazÃ¡s,A praga,Aa Intlo,Ab... Bas!,About A Short Film,Accused at 17,Achanak,Achoura,Adam & Eva,Afflatus,Afghan Knights,After the Third Bell,Aftermath: A Test of Love,Agatha,Agents of Change: Project Polymer,AibÃ´: GekijÃ´-ban IV,Akala mo...,Ako si Ninoy,All Flowers in Time,All My Life,All the Fists in the World Won't Save You Now,Almost Heaven,Alternate,Alyti,Am I,American Exorcist,American Short,Among Thieves,Amor que mata,An Idle Mind Is the Devil's Playground,An Unexpected Love,Anakiseuteu Anarchists,Angaaray,Annabelle: Creation,Annabellum: The Curse of Salem,Anomie,Anonymous Vol. 1 - The Dreamland Adventures the Mysteries of Atlantis,Anti Cupido,Any Way the Wind Blows,Apocalypse Road,Apostle,Apparition,Armugan,Aryan: Unbreakable,Asbury Park,Askin GÃ¶nÃ¼l Yazisi,Astro,Aswang: A Journey Into Myth,Attack of the Adult Babies,Aux,Awaken,Azaad Desh Ke Gulam,BIR,Bacao,Bakom Snart Ã¤r det lÃ¶rdag igen,Ban khang winyan,Bandwagon,Bangkok Dangerous,Bank Robber,Bannerghatta,Batman: Dead End,Bats in the Belfry,Bbuddah... Hoga Terra Baap,Be-seu-teu-sel-leo,Beat,Ben: The Movie,Bernard and Doris,Between the Trees,Bhoner: The Movie,Bigfoot,Birthday,Bit Players,Black Pond,Black Pumpkin,Blarghaaahrgarg,Blindsided,Blood Vow,Blue,Boarding School,Bob Roberts,Bobby,Body Bags,Bolse vita,Bonnie & Clyde vs. Dracula,Boogeyman Pop,"Boonie Bears, to the Rescue!",Boy Friend,Boy Meets Girl Stories #4: Geen antwoord,BoyTown,Boychick,Brightheart,Bun siu hai,Buster's Mal Heart,Buttf**ker,Caliche Run,Call Me Claus,Calling,Camp X-Ray: Ghosts of GuantÃ¡namo Bay,Captive Heart: The James Mink Story,Casablanca,Cassette,Caustic Zombies,Ce jour-lÃ,Cerro Torre: Schrei aus Stein,Cesta peklem,Chaco,Charlie St. Cloud,Cheun,Cheuuat gaawn chim,Chi bi: Jue zhan tian xia,Children of the Grave 2,Chingaari,Chong qing ai qing gan jue,"Chot [Aaj Isko, Kal Tereko]",Christian Blake,Christmas Twister,Christmas on Division Street,Chuck Steel: Night of the Trampires,Churuli,Circles,Class Savage,Client Seduction,CobraGator,Color-Blinded,Como Nascem os Anjos,Company K,Cowboy,Creature,Criss Angel Mindfreak,Criss Angel Supernatural,Crna ovca u restoranu za zombije,Crumbs,Cul de sac,Cumbia callera,Curse of Mesopotamia,Daava,Daddy Cross,Dagaalty,Dai zek lo,Dante's View,Dark Heart,Darkness Reigns,Darkwood Manor,Dawna of the Dead,Day 66,De zaak Alzheimer,Dead Again,Dead Ahead,Dead Bodies Everywhere,Dead of Winter,Deadtime,Dear Father,"Death, Deceit & Destiny Aboard the Orient Express",Demons,Den Japanska Illusionen,Den sorte celle,Der Traum vom SÃ¼den,Der Wien-Krimi: Blind ermittelt - Tod an der Donau,Der Zauber des Regenbogens,Detektor,Devil's Pond,Devot,Dibbuk,Die Mediocren,Die Waschmaschine,Die andere Frau,Die: Lianhua,Diljale,Dinner with Friends,DirektÃ¸ren for det hele,Distants,Distorted,Divar,Dok,Doli Saja Ke Rakhna,Dolphins,Don't Look Under the Bed,Don't Look in the Basement,Dontel papa,Doraemon: Nobita to fushigi kazetsukai,Doug's 1st Movie,Dr. Munshir Diary,Dream a Little Dream for Me,DristiBhram,Dubai,Duo ming jie chu,Easter Bunny Bloodbath,Eat Me,Eating Out,Eccezzziunale veramente - Capitolo secondo... me,Ecco fatto,Echoes of the Passed,Eddie,Ein Sack voll Geld,Ek: The Power of One,El ataque de los robots de nebulosa-5,El bar,El corazÃ³n de la tierra,El domini del sentits,El grito en el cielo,El que habita las alturas,El sol,El sueÃ±o de Ibiza,Elaan,Ellos Volvieron,Elsa y Fred,EmpanÃ¡ de pino,Encrucijada,End of the Line,Endoresu warutsu,Enemy Within,Enfants terribles,"English, August",Epoch: Evolution,Erakkorapu,Escape Room,Eskiya,Estrela Radiante,"Eternal Coma, 1994",Ex Abyssus,Exile,Exit 1

In [77]:
nombre_movie = "14 Days in Paradise"
rating = 5

similar_ratings = matr_corre[nombre_movie]*(rating-2.5)
recommendacion = similar_ratings.sort_values(ascending=False)

# printemoas la recomendacion
print(recommendacion[1:11].sort_values(ascending=False))

Titulo
Dolphins                                         2.5
Un homme                                         2.5
ShÃ´jo kakumei Utena: Adolescence mokushiroku    2.5
Udyanapalakan                                    2.5
Sirf Tum                                         2.5
Boy Friend                                       2.5
All My Life                                      2.5
Muskaan                                          2.5
Boy Meets Girl Stories #4: Geen antwoord         2.5
Shart: The Challenge                             2.5
Name: 14 Days in Paradise, dtype: float64
